In [124]:
import pandas as pd
import numpy as np
import warnings 
warnings.filterwarnings("ignore")

In [125]:
expedia_df = pd.read_csv('Expedia.csv')

In [126]:
expedia_df.head()

,Name,DateOfSearch,Checkin,Checkout,NumOfWeekendDays,Nights,District,Score,Rating,Review,Includes,Refundable,Price
0,The Hoxton Williamsburg,2023/3/1,2023/3/2,2023/3/3,0,1,Williamsburg,9.0,Wonderful,"(1,000 reviews)",NaN,Fully refundable,$347 total
1,Hampton Inn Manhattan/Times Square Central,2023/3/1,2023/3/2,2023/3/3,0,1,Theater District,8.6,Excellent,"(1,338 reviews)",Breakfast included,Fully refundable,$381 total
2,Cachet Boutique New York,2023/3/1,2023/3/2,2023/3/3,0,1,New York,7.6,Good,"(1,002 reviews)",NaN,0,$287 total
3,"PUBLIC, an Ian Schrager hotel",2023/3/1,2023/3/2,2023/3/3,0,1,New York,8.0,Very Good,"(1,447 reviews)",NaN,0,$446 total
4,Hampton Inn NY-JFK,2023/3/1,2023/3/2,2023/3/3,0,1,Queens,7.6,Good,"(1,216 reviews)",Breakfast included,0,$181 total


In [127]:
expedia_df.shape

(41368, 13)

In [128]:
expedia_df['WebIndex'] = (expedia_df.index % 100) + 1

In [129]:
expedia_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41368 entries, 0 to 41367
Data columns (total 14 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Name              41368 non-null  object
 1   DateOfSearch      41367 non-null  object
 2   Checkin           41367 non-null  object
 3   Checkout          41367 non-null  object
 4   NumOfWeekendDays  41367 non-null  object
 5   Nights            41367 non-null  object
 6   District          41367 non-null  object
 7   Score             40517 non-null  object
 8   Rating            40517 non-null  object
 9   Review            39051 non-null  object
 10  Includes          7780 non-null   object
 11  Refundable        41367 non-null  object
 12  Price             41367 non-null  object
 13  WebIndex          41368 non-null  int64 
dtypes: int64(1), object(13)
memory usage: 4.4+ MB


In [130]:
expedia_df.isna().sum()

Name                    0
DateOfSearch            1
Checkin                 1
Checkout                1
NumOfWeekendDays        1
Nights                  1
District                1
Score                 851
Rating                851
Review               2317
Includes            33588
Refundable              1
Price                   1
WebIndex                0
dtype: int64

In [131]:
##Dropping NaN rows from price == NaN
expedia_df = expedia_df.dropna(subset=['Price'])

In [132]:
##While scraping some of the reviews went to Rating column so move it back
mask = expedia_df['Review'].isnull()
expedia_df.loc[mask, 'Review'] = expedia_df.loc[mask, 'Rating']

In [133]:
##Cleaning the Review column
expedia_df['Review'] = expedia_df['Review'].str.replace('(', '')
expedia_df['Review'] = expedia_df['Review'].str.replace(')', '')
expedia_df['Review'] = expedia_df['Review'].str.replace('reviews', '')
expedia_df['Review'] = expedia_df['Review'].str.replace('Review', '')
expedia_df['Review'] = expedia_df['Review'].str.replace('review', '')
expedia_df['Review'] = expedia_df['Review'].str.replace(',', '')

In [134]:
expedia_df['Rating'] = expedia_df['Rating'].replace(to_replace='.*reviews.*', value=np.nan, regex=True)

In [135]:
expedia_df['Score'] = expedia_df['Score'].replace('Score', np.nan)
expedia_df['Rating'] = expedia_df['Rating'].replace('Rating', np.nan)

In [136]:
expedia_df['Nights'] = expedia_df['Nights'].str.replace('Nights', '')

In [137]:
expedia_df['Score'] = expedia_df['Score'].apply(pd.to_numeric)
expedia_df['Nights'] = expedia_df['Nights'].apply(pd.to_numeric)

In [138]:
##Function that find rating using score params
def map_score_to_rating(score):
    if score >= 9.5 and score <= 10:
        return 'Exceptional'
    elif score >= 9.0 and score < 9.5:
        return 'Excellent'
    elif score >= 8.5 and score < 9.0:
        return 'Wonderful'
    elif score >= 8.0 and score < 8.5:
        return 'Very Good'
    elif score >= 7.5 and score < 8.0:
        return 'Good'
    else:
        return 'Review score'

In [139]:
expedia_df['Rating'] = expedia_df.apply(lambda row: map_score_to_rating(row['Score']) if pd.isna(row['Rating']) else row['Rating'], axis=1)

In [140]:
expedia_df.replace({'Rating':{'Review score':5.0,'Good':6.0,'Very Good':7.0,'Excellent':8.0,
                        'Wonderful':9.0 ,'Exceptional':10.0}},inplace=True)

In [141]:
##Function that find score using rating params
def map_rating_to_score(rating):
    if rating == 'Exceptional': 
        return 9.5
    elif rating == 'Excellent':
        return 9.0
    elif rating == 'Wonderful':
        return 8.5
    elif rating == 'Very Good':
        return 8.0
    elif rating == 'Good':
        return 7.5
    else:
        return 6.0

In [142]:
expedia_df['Score'] = expedia_df.apply(lambda row: map_rating_to_score(row['Rating']) if pd.isna(row['Score']) else row['Score'], axis=1)

In [143]:
expedia_df.head()

,Name,DateOfSearch,Checkin,Checkout,NumOfWeekendDays,Nights,District,Score,Rating,Review,Includes,Refundable,Price,WebIndex
0,The Hoxton Williamsburg,2023/3/1,2023/3/2,2023/3/3,0,1.0,Williamsburg,9.0,9.0,1000,NaN,Fully refundable,$347 total,1
1,Hampton Inn Manhattan/Times Square Central,2023/3/1,2023/3/2,2023/3/3,0,1.0,Theater District,8.6,8.0,1338,Breakfast included,Fully refundable,$381 total,2
2,Cachet Boutique New York,2023/3/1,2023/3/2,2023/3/3,0,1.0,New York,7.6,6.0,1002,NaN,0,$287 total,3
3,"PUBLIC, an Ian Schrager hotel",2023/3/1,2023/3/2,2023/3/3,0,1.0,New York,8.0,7.0,1447,NaN,0,$446 total,4
4,Hampton Inn NY-JFK,2023/3/1,2023/3/2,2023/3/3,0,1.0,Queens,7.6,6.0,1216,Breakfast included,0,$181 total,5


In [144]:
expedia_df['Refundable'] = expedia_df['Refundable'].str.replace('Fully refundable','1')
expedia_df['Refundable'] = expedia_df['Refundable'].str.replace('Refundable','1')

In [145]:
expedia_df['Refundable'].unique()

array(['1', '0'], dtype=object)

In [146]:
##Cleaning Price column
expedia_df['Price'] = expedia_df['Price'].astype(str)
expedia_df['Price'] = expedia_df['Price'].str.replace('$', '')
expedia_df['Price'] = expedia_df['Price'].str.replace('total', '')
expedia_df['Price'] = expedia_df['Price'].str.replace(',', '')
expedia_df['Price'] = expedia_df['Price'].str.replace('Price', '')

In [147]:
##Cleaning Includes column
expedia_df['Includes'] = expedia_df['Includes'].replace('0', np.nan)
expedia_df['Includes'] = expedia_df['Includes'].replace('Includes', np.nan)

In [148]:
##OneHotEncoding for Includes and District columns
expedia_df = pd.get_dummies(expedia_df, columns = ['Includes'])
expedia_df = pd.get_dummies(expedia_df, columns = ['District'])

In [149]:
# Changing the dates columns to datetime format
expedia_df['DateOfSearch'] = pd.to_datetime(expedia_df['DateOfSearch'],errors='coerce', format='%Y-%m-%d')
expedia_df['Checkin'] = pd.to_datetime(expedia_df['Checkin'],errors='coerce', format='%Y-%m-%d')
expedia_df['Checkout'] = pd.to_datetime(expedia_df['Checkout'],errors='coerce', format='%Y-%m-%d')

In [150]:
##Creating new columns of months and days separately for each date
expedia_df['SnapShotMonth'] = expedia_df['DateOfSearch'].dt.month.astype(str)
expedia_df['SnapShotDay'] = expedia_df['DateOfSearch'].dt.day.astype(str)
expedia_df['CheckinMonth'] = expedia_df['Checkin'].dt.month.astype(str)
expedia_df['CheckinDay'] = expedia_df['Checkin'].dt.day.astype(str)
expedia_df['CheckoutMonth'] = expedia_df['Checkout'].dt.month.astype(str)
expedia_df['CheckoutDay'] = expedia_df['Checkout'].dt.day.astype(str)

In [151]:
##Creating TTT and LOS columns
expedia_df['TTT'] = (expedia_df['Checkin'] - expedia_df['DateOfSearch']).dt.days
expedia_df['LOS'] = expedia_df['Nights'] + 1

In [152]:
##Cleaning NumOfWeekendDays column
expedia_df['NumOfWeekendDays'] = expedia_df['NumOfWeekendDays'].str.replace('NumOfWeekendDays', '')

In [153]:
expedia_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41367 entries, 0 to 41366
Data columns (total 39 columns):
 #   Column                             Non-Null Count  Dtype         
---  ------                             --------------  -----         
 0   Name                               41367 non-null  object        
 1   DateOfSearch                       41365 non-null  datetime64[ns]
 2   Checkin                            41365 non-null  datetime64[ns]
 3   Checkout                           41365 non-null  datetime64[ns]
 4   NumOfWeekendDays                   41367 non-null  object        
 5   Nights                             41365 non-null  float64       
 6   Score                              41367 non-null  float64       
 7   Rating                             41367 non-null  float64       
 8   Review                             40517 non-null  object        
 9   Refundable                         41367 non-null  object        
 10  Price                             

In [154]:
expedia_df = expedia_df.dropna(subset=['Review'])
##Dropping the full date
expedia_df.drop('DateOfSearch', inplace=True, axis=1)
expedia_df.drop('Checkin', inplace=True, axis=1)
expedia_df.drop('Checkout', inplace=True, axis=1)
expedia_df.drop('District_District',axis = 1 , inplace = True)

In [155]:
##Dropping NaN values again
expedia_df = expedia_df.dropna()

In [156]:
expedia_df.isna().sum()

Name                                 0
NumOfWeekendDays                     0
Nights                               0
Score                                0
Rating                               0
Review                               0
Refundable                           0
Price                                0
WebIndex                             0
Includes_Airport shuttle included    0
Includes_Breakfast included          0
Includes_Kitchen                     0
Includes_Pool                        0
District_Bronx                       0
District_Brooklyn                    0
District_Central New York City       0
District_Chelsea                     0
District_Entire home                 0
District_Flushing                    0
District_Greenwich Village           0
District_Long Island City            0
District_New York                    0
District_Queens                      0
District_SoHo                        0
District_Theater District            0
District_Upper West Side 

In [157]:
##Converting columns to numerical
expedia_df['NumOfWeekendDays'] = expedia_df['NumOfWeekendDays'].apply(pd.to_numeric)
expedia_df['Review'] = expedia_df['Review'].apply(pd.to_numeric)
expedia_df['Refundable'] = expedia_df['Refundable'].apply(pd.to_numeric)
expedia_df['Price'] = expedia_df['Price'].apply(pd.to_numeric)
expedia_df['SnapShotMonth'] = expedia_df['SnapShotMonth'].apply(pd.to_numeric)
expedia_df['SnapShotDay'] = expedia_df['SnapShotDay'].apply(pd.to_numeric)
expedia_df['CheckinMonth'] = expedia_df['CheckinMonth'].apply(pd.to_numeric)
expedia_df['CheckinDay'] = expedia_df['CheckinDay'].apply(pd.to_numeric)
expedia_df['CheckoutMonth'] = expedia_df['CheckoutMonth'].apply(pd.to_numeric)
expedia_df['CheckoutDay'] = expedia_df['CheckoutDay'].apply(pd.to_numeric)

In [158]:
##Creating new column that check price per night
expedia_df['TotalPrice'] = (expedia_df['Price'] * expedia_df['Nights']).round()

In [160]:
expedia_df['TotalPrice'] = expedia_df['TotalPrice'].apply(pd.to_numeric)

In [161]:
expedia_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 40515 entries, 0 to 41366
Data columns (total 36 columns):
 #   Column                             Non-Null Count  Dtype  
---  ------                             --------------  -----  
 0   Name                               40515 non-null  object 
 1   NumOfWeekendDays                   40515 non-null  int64  
 2   Nights                             40515 non-null  float64
 3   Score                              40515 non-null  float64
 4   Rating                             40515 non-null  float64
 5   Review                             40515 non-null  int64  
 6   Refundable                         40515 non-null  int64  
 7   Price                              40515 non-null  int64  
 8   WebIndex                           40515 non-null  int64  
 9   Includes_Airport shuttle included  40515 non-null  uint8  
 10  Includes_Breakfast included        40515 non-null  uint8  
 11  Includes_Kitchen                   40515 non-null  uin

In [162]:
expedia_df.head()

,Name,NumOfWeekendDays,Nights,Score,Rating,Review,Refundable,Price,WebIndex,Includes_Airport shuttle included,...,District_Williamsburg,SnapShotMonth,SnapShotDay,CheckinMonth,CheckinDay,CheckoutMonth,CheckoutDay,TTT,LOS,TotalPrice
0,The Hoxton Williamsburg,0,1.0,9.0,9.0,1000,1,347,1,0,...,1,3.0,1.0,3.0,2.0,3.0,3.0,1.0,2.0,347.0
1,Hampton Inn Manhattan/Times Square Central,0,1.0,8.6,8.0,1338,1,381,2,0,...,0,3.0,1.0,3.0,2.0,3.0,3.0,1.0,2.0,381.0
2,Cachet Boutique New York,0,1.0,7.6,6.0,1002,0,287,3,0,...,0,3.0,1.0,3.0,2.0,3.0,3.0,1.0,2.0,287.0
3,"PUBLIC, an Ian Schrager hotel",0,1.0,8.0,7.0,1447,0,446,4,0,...,0,3.0,1.0,3.0,2.0,3.0,3.0,1.0,2.0,446.0
4,Hampton Inn NY-JFK,0,1.0,7.6,6.0,1216,0,181,5,0,...,0,3.0,1.0,3.0,2.0,3.0,3.0,1.0,2.0,181.0


In [165]:
expedia_df

,Name,NumOfWeekendDays,Nights,Score,Rating,Review,Refundable,Price,WebIndex,Includes_Airport shuttle included,...,District_Williamsburg,SnapShotMonth,SnapShotDay,CheckinMonth,CheckinDay,CheckoutMonth,CheckoutDay,TTT,LOS,TotalPrice
0,The Hoxton Williamsburg,0,1.0,9.0,9.0,1000,1,347,1,0,...,1,3.0,1.0,3.0,2.0,3.0,3.0,1.0,2.0,347.0
1,Hampton Inn Manhattan/Times Square Central,0,1.0,8.6,8.0,1338,1,381,2,0,...,0,3.0,1.0,3.0,2.0,3.0,3.0,1.0,2.0,381.0
2,Cachet Boutique New York,0,1.0,7.6,6.0,1002,0,287,3,0,...,0,3.0,1.0,3.0,2.0,3.0,3.0,1.0,2.0,287.0
3,"PUBLIC, an Ian Schrager hotel",0,1.0,8.0,7.0,1447,0,446,4,0,...,0,3.0,1.0,3.0,2.0,3.0,3.0,1.0,2.0,446.0
4,Hampton Inn NY-JFK,0,1.0,7.6,6.0,1216,0,181,5,0,...,0,3.0,1.0,3.0,2.0,3.0,3.0,1.0,2.0,181.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41362,Fitzpatrick Manhattan Hotel,2,5.0,8.8,8.0,1001,1,380,63,0,...,0,3.0,3.0,4.0,1.0,4.0,6.0,29.0,6.0,1900.0
41363,Hilton Club West 57th Street New York,2,5.0,9.2,9.0,223,0,395,64,0,...,0,3.0,3.0,4.0,1.0,4.0,6.0,29.0,6.0,1975.0
41364,Roxy Hotel New York,2,5.0,9.0,9.0,1000,1,512,65,0,...,0,3.0,3.0,4.0,1.0,4.0,6.0,29.0,6.0,2560.0
41365,Equinox Hotel New York,2,5.0,9.0,9.0,304,0,908,66,0,...,0,3.0,3.0,4.0,1.0,4.0,6.0,29.0,6.0,4540.0


In [166]:
##Saving the clean data to new csv
expedia_df.to_csv('Expedia_cleaned.csv',index=False)